In [1]:
#Question 1.2
import numpy as np
import pandas as pd
import requests
import quandl as ql
from datetime import datetime
import matplotlib.pyplot as plt
import statsmodels.formula.api as sm


ModuleNotFoundError: No module named 'quandl'

In [ ]:
api_key = 'jNnWDGnTMrGUz5LriNqn'
apiurl = f'https://demo.quandl.com/api/v3/datasets/OPEC/ORB.csv?api_key={api_key}'
ETH = requests.get(apiurl).json()['Data']['Data']
Yt_1 = pd.DataFrame(ETH)[['time', 'open']].set_index('time')

In [ ]:
ToTimeStamp = ETH[0]['time']
apiurl = f'https://min-api.cryptocompare.com/data/v2/histoday?fsym=ETH&tsym=USD&limit=2000&toTs={ToTimeStamp}&api_key={api_key}'
ETH1 = requests.get(apiurl).json()['Data']['Data']
Yt_2 = pd.DataFrame(ETH1)[['time', 'open']].set_index('time')

In [ ]:
ToTimeStamp = ETH1[0]['time']
apiurl = f'https://min-api.cryptocompare.com/data/v2/histoday?fsym=ETH&tsym=USD&limit=2000&toTs={ToTimeStamp}&api_key={api_key}'
ETH2 = requests.get(apiurl).json()['Data']['Data']
Yt_3 = pd.DataFrame(ETH2)[['time', 'open']].set_index('time')
Yt_data = pd.concat([Yt_3,Yt_2,Yt_1])
Yt_data.index = pd.to_datetime(Yt_data.index, unit = 's')
Yt_data = Yt_data[ Yt_data.index > '2010-07-17'] 
Yt_data.reset_index(inplace=True)
Yt_data.rename(columns ={'time':'Date'}, inplace='True')
Yt_data.dtypes

In [ ]:
ql.ApiConfig.api_key = "jNnWDGnTMrGUz5LriNqn"
Xt = pd.DataFrame(ql.get("MULTPL/SP500_REAL_PRICE_MONTH"))
Xt = Xt[ Xt.index > '2010-07-17'] 
Xt.reset_index(inplace=True)
Xt.dtypes

In [ ]:
df = Xt.merge(Yt_data, how ='left', on='Date')
df.rename(columns ={'Value':'Xt','open':'Yt'}, inplace='True')
df

In [ ]:
plt.plot(df.Xt,df.Yt,'go')

In [ ]:
beta = ((np.multiply(df.Yt, df.Xt)).mean()-np.mean(df.Xt)*np.mean(df.Yt))/((np.multiply(df.Xt,df.Xt)).mean()-np.mean(df.Xt)*np.mean(df.Xt))
alpha =np.mean(df.Yt)-beta*np.mean(df.Xt)
print("alpha =",alpha, "\nbeta =",beta)

In [ ]:
type((np.multiply(df.Yt, df.Xt)))
y_hat = alpha + np.multiply(beta,df.Xt)
L = np.sum(np.multiply(df.Yt - y_hat,df.Yt - y_hat))
L

In [ ]:
result = sm.ols('Yt~Xt',data=df).fit()
print(result.summary())

In [ ]:
def  cal_cost(a,b,x,y):
        n = len(y)
        pred = a + np.multiply(b,x)
        cost = (1/2*n) * np.sum(np.square(pred-y))
        return cost

In [ ]:
itr = 1000
alpha = 2000
n = len(df.Yt)

loss_his = np.zeros((itr,1))
alpha_his = np.zeros((itr,1))

for i in range(itr):
        pred = alpha + np.multiply(beta,df.Xt)
        res= df.Yt-pred
        res_sum = sum(res)
        gradient = -(1/n)*0.1*res_sum
 
        alpha = alpha - gradient
    
        loss_his[i] = cal_cost(alpha,beta,df.Xt,df.Yt)
        alpha_his[i] = alpha
    
plt.plot(alpha_his,loss_his,'go')

In [2]:
#Question 4
import requests
import json
import pandas as pd
import datetime

In [3]:
lim = '30'
pair = ['ETH','BSV','BTC','XRP','LTC','DOGE','BCH','BNB','XTZ','EOS','XLM']
coins = pd.DataFrame()

In [4]:
api_Key = "f65355cfcf50e300e5fcf3123382110667b44175c066ed2202cf0ae56931e8e7"

In [5]:
for i in pair:
    URL_A = 'https://min-api.cryptocompare.com/data/histoday?fsym='+i+'&tsym=USD&limit='+lim+ '&api_key='
    URL = URL_A + api_Key
    data = requests.get(URL)
    json_data = data.json()
    table = pd.json_normalize(json_data, 'Data').set_index('time')
    table.index = pd.to_datetime(table.index ,unit='s')
    coins = pd.concat([coins, table.high], axis=1)
coins.columns = pair

In [6]:
coins.columns = pair
coins = coins[pair]
coins

,ETH,BSV,BTC,XRP,LTC,DOGE,BCH,BNB,XTZ,EOS,XLM
2022-07-07 00:00:00,1253.43,58.70,21840.22,0.3452,52.58,0.07117,112.43,243.60,1.593,1.0370,0.1154
2022-07-08 00:00:00,1274.65,57.74,22377.17,0.3580,53.96,0.07338,113.19,247.47,1.598,1.0680,0.1183
2022-07-09 00:00:00,1232.47,57.37,21942.80,0.3474,54.77,0.07011,113.07,244.89,1.654,1.0480,0.1150
2022-07-10 00:00:00,1219.06,55.16,21591.09,0.3448,55.01,0.06947,111.52,243.27,1.663,1.0390,0.1129
2022-07-11 00:00:00,1170.23,52.17,20852.04,0.3287,52.19,0.06724,107.30,234.89,1.708,1.0070,0.1078
2022-07-12 00:00:00,1097.18,50.57,20036.49,0.3177,49.31,0.06302,100.97,228.02,1.622,0.9490,0.1034
2022-07-13 00:00:00,1117.22,52.82,20274.83,0.3253,49.65,0.06227,102.60,230.74,1.522,0.9561,0.1055
2022-07-14 00:00:00,1211.87,53.95,20869.51,0.3336,51.69,0.06276,104.11,239.93,1.581,0.9762,0.1072
2022-07-15 00:00:00,1286.06,53.53,21185.15,0.3484,52.72,0.06438,106.98,242.62,1.620,0.9927,0.1085
2022-07-16 00:00:00,1400.05,54.71,21574.75,0.3560,55.49,0.06481,110.66,251.46,1.638,1.0140,0.1102


In [ ]:
coins_lagged=coins.copy()
trailing_window_size = 1
for window in range(1, trailing_window_size +1):
    shifted = coins.shift(window)
    shifted.columns = [x + "_lag" + str(window) for x in coins.columns]
    
    coins_lagged = pd.concat((coins_lagged, shifted), axis=1)
coins_lagged = coins_lagged.dropna()

In [ ]:
coins_lagged

In [ ]:
coins_lagged.to_csv('cryto_pricedata.csv', index_label='time')
df = pd.read_csv('cryto_pricedata.csv', index_col='time')

In [ ]:
df.index = pd.to_datetime(df.index)
df.index = df.index.normalize()

In [ ]:
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

In [ ]:
sc_x = StandardScaler()
df_scaled = pd.DataFrame(sc_x.fit_transform(df), index=df.index, columns=df.columns)
plt.plot(df_scaled.index, df_scaled['BTC'], color='green')
plt.plot(df_scaled.index, df_scaled['ETH'], color='red')
plt.plot(df_scaled.index, df_scaled['DOGE'], color='blue')

In [ ]:
split = int(int(lim)/2)

In [ ]:
Ytrain = pd.DataFrame(df_scaled[:split]['BTC'])
Ytest = pd.DataFrame(df_scaled[split:]['BTC'])
X = df_scaled[['BTC_lag1','BSV_lag1', 'ETH_lag1','BCH_lag1','LTC_lag1','EOS_lag1','XRP_lag1','BNB_lag1','XTZ_lag1','XLM_lag1','DOGE_lag1']]

In [ ]:
from sklearn.linear_model import ElasticNet
import numpy as np
en = ElasticNet(alpha=0.1, normalize=False)
en.fit(X[:split], Ytrain)
coef = list(en.coef_)
coef

In [ ]:
count = 0
for i in X.columns:
    print(i, ':', coef[count])
    count = count + 1

In [ ]:
BTC_fitted = X.dot(coef)
BTC_fitted

In [ ]:
BTC_actual = df_scaled[['BTC']]
BTC_actual

In [ ]:
plt.plot(df_scaled.index, BTC_fitted, color='red')
plt.plot(df_scaled.index, BTC_actual , color='green')

In [ ]:
y = BTC_actual
X = BTC_fitted
y = y.values.reshape(-1,1)
X = X.values.reshape(-1,1)
sample_size = len(df_scaled)
squared_residuals = np.square(X-y)
sum_squared_residuals = np.sum(squared_residuals)
sum_squared_residuals

In [ ]:
MSE = sum_squared_residuals/sample_size
MSE

In [ ]:
from sklearn.linear_model import LinearRegression
X = df_scaled[['BTC_lag1']]
y = df_scaled['BTC']
y = y.values.reshape(-1,1)
X = X.values.reshape(-1,1)
lr = LinearRegression()

In [ ]:
lr.fit(X,y)
print(lr.intercept_, lr.coef_)

In [ ]:
forecast_BTC = lr.predict(X)

In [ ]:
squared_residuals = np.square(forecast_BTC-y)

In [ ]:
MSE = (np.sum(squared_residuals))/len(y)
MSE